In [1]:
import numpy as np
from numpy import *
from cvxopt import matrix, spmatrix, umfpack, mul, sqrt, div
from scipy.stats.stats import pearsonr

In [2]:
# FMD step
h = deg2rad(1)                      # in radians

In [3]:
# Coordinates
Theta,Phi = linspace(h,pi-h,179),linspace(0,2*pi,361)
N,M       = len(Theta),len(Phi)
Ph,Th     = np.meshgrid(Phi, Theta)       # Ph,Th Lon-Lat Grid [N,M]
# Sin,Cos,Tan
sth,cth,tth = sin(Th),cos(Th),tan(Th)

In [10]:
    # Equation factors
    a = (-2/h**2)*(1+(sth)**-2)
    b = ( 1/h**2)+(2*h*tth)**-1
    c = ( 1/h**2)-(2*h*tth)**-1
    d = ( 1/h**2)*(sth)**-2
    P = ( 2/h**2)
    #b[N-1],c[0] = 0,0
    b[0],b[N-1] = P,0
    c[0],c[N-1] = 0,P
    av = matrix(a,(1,N*M))
    bv = matrix(b,(1,N*M))
    cv = matrix(c,(1,N*M))
    dv = matrix(d,(1,N*M))
    print c

[[   0.            0.            0.         ...    0.
     0.            0.        ]
 [2462.43812292 2462.43812292 2462.43812292 ... 2462.43812292
  2462.43812292 2462.43812292]
 [2736.17204975 2736.17204975 2736.17204975 ... 2736.17204975
  2736.17204975 2736.17204975]
 ...
 [3829.44065027 3829.44065027 3829.44065027 ... 3829.44065027
  3829.44065027 3829.44065027]
 [4103.1745771  4103.1745771  4103.1745771  ... 4103.1745771
  4103.1745771  4103.1745771 ]
 [6565.61270002 6565.61270002 6565.61270002 ... 6565.61270002
  6565.61270002 6565.61270002]]


In [11]:
    # A Matrix
    MD = spmatrix(av,            range(N*M),       range(N*M)                    )
    B  = spmatrix(bv[0:N*M-1],   range(N*M-1),     range(1,N*M),        (N*M,N*M))
    C  = spmatrix(cv[1:N*M],     range(1,N*M),     range(N*M-1),        (N*M,N*M))
    DU = spmatrix(dv[0:N*M-(N)], range(N*M-N),     range(N,N*M),        (N*M,N*M))
    EU = spmatrix(dv[0:N],       range(N),         range(N*M-2*N,N*M-N),(N*M,N*M))
    ED = spmatrix(dv[N:N*M],     range(N,N*M),     range(N*M-N),        (N*M,N*M))
    DD = spmatrix(dv[N*M-N:N*M], range(N*M-N,N*M), range(N,2*N),        (N*M,N*M))
    A = MD+B+C+DU+DD+EU+ED

In [12]:
    R = 3485e+0
    Wr = (2/R**2)*cth
    # Inversion
    #R = 3485e+0 
    V = matrix(((-R**2)*Wr),(N*M,1))
    umfpack.linsolve(A,V)
    Y = matrix(V,(N,M))
    
    print Y

[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00  2.95e+00 ... ]
[ 2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00 ... ]
[ 2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00 ... ]
[ 2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00 ... ]
[ 2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00  2.94e+00 ... ]
[ 2.93e+00  2.93e+00  2.93e+00  2.93e+00  2.93e+00  2.93e+00  2.93e+00 ... ]
[ 2.93e+00  2.93e+00  2.93e+00  2.93e+00  2.93e+00  2.93e+00  2.93e+00 ... ]

In [8]:
N,M = 3,3
I = np.ones(N*M)

av,bv,cv,dv = I,I,I,I

MD = spmatrix(av,            range(N*M),       range(N*M)                    )
B  = spmatrix(bv[0:N*M-1],   range(N*M-1),     range(1,N*M),        (N*M,N*M))
C  = spmatrix(cv[1:N*M],     range(1,N*M),     range(N*M-1),        (N*M,N*M))
DU = spmatrix(dv[0:N*M-(N)], range(N*M-N),     range(N,N*M),        (N*M,N*M))
EU = spmatrix(dv[0:N],       range(N),         range(N*M-2*N,N*M-N),(N*M,N*M))
ED = spmatrix(dv[N:N*M],     range(N,N*M),     range(N*M-N),        (N*M,N*M))
DD = spmatrix(dv[N*M-N:N*M], range(N*M-N,N*M), range(N,2*N),        (N*M,N*M))
A = MD+B+C+DU+DD+EU+ED
print A

[ 1.00e+00  1.00e+00     0      2.00e+00     0         0         0     ... ]
[ 1.00e+00  1.00e+00  1.00e+00     0      2.00e+00     0         0     ... ]
[    0      1.00e+00  1.00e+00  1.00e+00     0      2.00e+00     0     ... ]
[ 1.00e+00     0      1.00e+00  1.00e+00  1.00e+00     0      1.00e+00 ... ]
[    0      1.00e+00     0      1.00e+00  1.00e+00  1.00e+00     0     ... ]
[    0         0      1.00e+00     0      1.00e+00  1.00e+00  1.00e+00 ... ]
[    0         0         0      2.00e+00     0      1.00e+00  1.00e+00 ... ]
[    0         0         0         0      2.00e+00     0      1.00e+00 ... ]
[    0         0         0         0         0      2.00e+00     0     ... ]

